# 🧠 Classificação: Predição de Conversão de Ofertas

Este notebook tem como objetivo construir um modelo de classificação que auxilie o iFood a **prever se um cliente completará uma oferta recebida**. Essa predição pode ser usada para **personalizar campanhas**, **priorizar canais de comunicação** e **aumentar a taxa de conversão das promoções**.

## 🧩 Problema

Dado um conjunto de dados com informações sobre clientes, características das ofertas e eventos registrados durante a campanha (como *"offer received"*, *"offer viewed"*, *"offer completed"*, *"transaction"*, etc), o objetivo é:

> **Prever se uma oferta será completada pelo cliente após ter sido recebida.**

---

## 📚 Etapas da modelagem

1. **Seleção e preparação dos dados**
   - Filtragem de eventos do tipo `"offer received"`
   - Enriquecimento com dados do cliente e detalhes da oferta
   - Criação da variável alvo: `completed` (1 para ofertas completadas, 0 caso contrário)

2. **Engenharia de atributos**
   - Conversão de variáveis categóricas para numéricas (One-Hot Encoding ou Label Encoding)
   - Transformação de datas
   - Criação de grupos etários e faixas de limite de crédito

3. **Divisão dos dados**
   - Separação entre conjunto de treino e teste (ex: 80/20)

4. **Treinamento de modelos**
   - Modelos como `RandomForestClassifier`, `LogisticRegression` e `XGBoost` podem ser utilizados
   - Ajuste de hiperparâmetros (opcional)

5. **Avaliação**
   - Métricas: *Accuracy*, *Precision*, *Recall*, *F1-score*
   - Análise de matriz de confusão e balanceamento das classes

6. **Análise dos resultados**
   - Interpretação de importância dos atributos
   - Discussão sobre viabilidade de uso do modelo em campanhas reais

---

## 🎯 Resultado Esperado

O modelo será capaz de auxiliar o time de marketing a **decidir qual cliente deve receber determinada oferta**, maximizando o retorno das campanhas promocionais.

In [1]:
import sys
import os

# Obtém o caminho absoluto do diretório 'src'
src_path = os.path.abspath("../")

if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
import pandas as pd

## 1. Leitura, seleção e preparação dos dados

In [3]:
# Leitura do arquivo parquet gerado pelo Spark
df = pd.read_parquet("../data/processed/full_data")

In [4]:
df.head()

,account_id,event,time_since_test_start,value_amount,value_reward,offer_id,age,credit_card_limit,gender,registered_on,...,age_group,credit_limit_bucket,discount_value,duration,min_value,offer_type,channels_mobile,channels_email,channels_social,channels_web
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0.0,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,75,100000.0,F,20170509,...,Boomers (60+),Very High (> 60k),5.0,7.0,5.0,bogo,1.0,1.0,0.0,1.0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0.0,NaN,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,118,63000.0,unknown,20170804,...,Boomers (60+),Very High (> 60k),5.0,10.0,20.0,discount,0.0,1.0,0.0,1.0
2,e2127556f4f64592b11af22de27a7932,offer received,0.0,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,68,70000.0,M,20180426,...,Boomers (60+),Very High (> 60k),2.0,7.0,10.0,discount,1.0,1.0,0.0,1.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0.0,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,118,63000.0,unknown,20170925,...,Boomers (60+),Very High (> 60k),2.0,10.0,10.0,discount,1.0,1.0,1.0,1.0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0.0,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,118,63000.0,unknown,20171002,...,Boomers (60+),Very High (> 60k),10.0,5.0,10.0,bogo,1.0,1.0,1.0,1.0


In [5]:
df.shape

(306534, 21)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   account_id             306534 non-null  object 
 1   event                  306534 non-null  object 
 2   time_since_test_start  306534 non-null  float64
 3   value_amount           138953 non-null  float64
 4   value_reward           33579 non-null   float64
 5   offer_id               167581 non-null  object 
 6   age                    306534 non-null  int64  
 7   credit_card_limit      306534 non-null  float64
 8   gender                 306534 non-null  object 
 9   registered_on          306534 non-null  object 
 10  birth_year             306534 non-null  int64  
 11  age_group              306534 non-null  object 
 12  credit_limit_bucket    306534 non-null  object 
 13  discount_value         167581 non-null  float64
 14  duration               167581 non-nu

### 1.1 Seleção das ofertas recebidas e completadas e criação da label (target)

In [7]:
offers_df = df[df["event"].isin(["offer received", "offer completed"])].copy()

In [8]:
offers_df["label"] = (offers_df["event"] == "offer completed").astype(int)

### 1.2 Verificação do balanceamento dos dados

In [9]:
pd.concat([offers_df['label'].value_counts(), 
          offers_df['label'].value_counts(normalize=True)*100], axis=1)

,count,proportion
label,,
0,76277,69.433622
1,33579,30.566378


In [10]:
### 1.3 Seleção das features

In [11]:
list(offers_df.columns)

['account_id',
 'event',
 'time_since_test_start',
 'value_amount',
 'value_reward',
 'offer_id',
 'age',
 'credit_card_limit',
 'gender',
 'registered_on',
 'birth_year',
 'age_group',
 'credit_limit_bucket',
 'discount_value',
 'duration',
 'min_value',
 'offer_type',
 'channels_mobile',
 'channels_email',
 'channels_social',
 'channels_web',
 'label']

In [12]:
# 🧱 Mantém apenas colunas relevantes
features = [
    "age", "gender", "credit_card_limit",
    "birth_year", "age_group", "credit_limit_bucket", "offer_type",
    "discount_value", "duration", "min_value", "channels_email",
    "channels_mobile", "channels_social", "channels_web"
]
extra_columns = [col for col in df.columns if col not in features]
print("❌ Colunas extras:", extra_columns)

❌ Colunas extras: ['account_id', 'event', 'time_since_test_start', 'value_amount', 'value_reward', 'offer_id', 'registered_on']


## 2. Divisão dos dados em treino e teste

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = offers_df[features]
y = offers_df["label"]

# Primeiro: separa em treino (60%) e resto (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)

# Depois: separa o resto em validação (20%) e teste (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

# Confere as proporções
print(f"Treino: {len(X_train)}")
print(f"Validação: {len(X_val)}")
print(f"Teste: {len(X_test)}")

Treino: 65913
Validação: 21971
Teste: 21972


## 3. Pipeline de treinamento preparação dos dados (dados categoricos e dados numericos)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [16]:
# Listas de colunas
categorical_features = ['gender', 'age_group', 'credit_limit_bucket', 'offer_type']
numerical_features = ['age', 'credit_card_limit', 'duration', 'min_value', 'discount_value']

categorical_features = X.select_dtypes(include="object").columns.tolist()
numerical_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

In [17]:
# 🔧 Preprocessadores
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numerical_features),
    ("cat", categorical_pipeline, categorical_features)
])


## 4. Definição dos modelos e parâmetros

In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [19]:
models = {
    "LogisticRegression": (
        LogisticRegression(max_iter=2000, solver="liblinear"),
        {
            "model__C": [0.01, 0.1, 1, 10],
            "model__class_weight": [None, "balanced"]
        }
    ),
    "RandomForest": (
        RandomForestClassifier(random_state=42),
        {
            "model__n_estimators": [100, 200],
            "model__max_depth": [10, 20, None],
            "model__class_weight": [None, "balanced"]
        }
    ),
    "XGBoost": (
        XGBClassifier(eval_metric="logloss", random_state=42),
        {
            "model__n_estimators": [100, 200],
            "model__max_depth": [3, 5, 7],
            "model__learning_rate": [0.1, 0.01],
            "model__subsample": [0.8, 1.0],
            "model__colsample_bytree": [0.8, 1.0]
        }
    ),
    "LightGBM": (
        LGBMClassifier(random_state=42),
        {
            "model__n_estimators": [100, 200],
            "model__max_depth": [10, 20, -1],
            "model__learning_rate": [0.1, 0.05],
            "model__class_weight": [None, "balanced"]
        }
    )
}

## 5. Avaliação dos modelos

🎯 F1 Score – A melhor candidata para este caso

✅ Por que F1 Score?
- Combina precisão e recall: é útil quando falsos positivos e falsos negativos têm custos relevantes.
- Se você prever que um cliente vai completar uma oferta e ele não completa (falso positivo), você pode gastar uma verba de marketing à toa.
- Se você não prever que o cliente completaria (falso negativo), você perde uma oportunidade de engajamento.
- F1 Score = 2 × (Precision × Recall) / (Precision + Recall)
- Ele equilibra os dois, sendo especialmente útil em bases desbalanceadas.

In [ ]:
# 🔹 7. Treinamento e avaliação
results = {}

for name, (model, param_grid) in models.items():
    print(f"\n🔍 Treinando modelo: {name}")
    
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model)
    ])

    grid = GridSearchCV(pipe, param_grid, scoring="f1", cv=3, n_jobs=-1)
    grid.fit(X_train, y_train)

    # 🔎 Validação
    print(f"✅ Melhores parâmetros ({name}): {grid.best_params_}")
    val_pred = grid.predict(X_val)
    print("\n📊 Avaliação no VALIDAÇÃO:")
    print(classification_report(y_val, val_pred))

    # 🧪 Teste
    test_pred = grid.predict(X_test)
    print("📊 Avaliação no TESTE:")
    print(classification_report(y_test, test_pred))

    # 🔒 Guarda os resultados
    results[name] = {
        "best_estimator": grid.best_estimator_,
        "val_report": classification_report(y_val, val_pred, output_dict=True),
        "test_report": classification_report(y_test, test_pred, output_dict=True)
    }


🔍 Treinando modelo: LogisticRegression
✅ Melhores parâmetros (LogisticRegression): {'model__C': 1, 'model__class_weight': 'balanced'}

📊 Avaliação no VALIDAÇÃO:
              precision    recall  f1-score   support

           0       0.86      0.47      0.61     15255
           1       0.41      0.83      0.55      6716

    accuracy                           0.58     21971
   macro avg       0.64      0.65      0.58     21971
weighted avg       0.72      0.58      0.59     21971

📊 Avaliação no TESTE:
              precision    recall  f1-score   support

           0       0.86      0.47      0.61     15256
           1       0.41      0.83      0.55      6716

    accuracy                           0.58     21972
   macro avg       0.64      0.65      0.58     21972
weighted avg       0.72      0.58      0.59     21972


🔍 Treinando modelo: RandomForest


/usr/local/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


✅ Melhores parâmetros (RandomForest): {'model__class_weight': 'balanced', 'model__max_depth': 10, 'model__n_estimators': 200}

📊 Avaliação no VALIDAÇÃO:
              precision    recall  f1-score   support

           0       0.88      0.43      0.57     15255
           1       0.40      0.87      0.55      6716

    accuracy                           0.56     21971
   macro avg       0.64      0.65      0.56     21971
weighted avg       0.73      0.56      0.57     21971

📊 Avaliação no TESTE:
              precision    recall  f1-score   support

           0       0.88      0.43      0.57     15256
           1       0.40      0.87      0.55      6716

    accuracy                           0.56     21972
   macro avg       0.64      0.65      0.56     21972
weighted avg       0.73      0.56      0.57     21972


🔍 Treinando modelo: XGBoost


/home/nicksson/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:51:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/nicksson/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:51:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/nicksson/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:51:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/nicksson/.local/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [00:51:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/nicksson/.local/lib/python3.11/site-packages/xgboo

✅ Melhores parâmetros (XGBoost): {'model__colsample_bytree': 1.0, 'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__n_estimators': 200, 'model__subsample': 0.8}

📊 Avaliação no VALIDAÇÃO:
              precision    recall  f1-score   support

           0       0.69      0.93      0.79     15255
           1       0.27      0.06      0.09      6716

    accuracy                           0.67     21971
   macro avg       0.48      0.49      0.44     21971
weighted avg       0.56      0.67      0.58     21971

📊 Avaliação no TESTE:
              precision    recall  f1-score   support

           0       0.69      0.93      0.79     15256
           1       0.27      0.06      0.10      6716

    accuracy                           0.66     21972
   macro avg       0.48      0.49      0.44     21972
weighted avg       0.56      0.66      0.58     21972


🔍 Treinando modelo: LightGBM


In [ ]:
## 